In [1]:
!pip install transformers datasets bertviz -q

In [16]:
with open('data/spam.txt', encoding='cp1251') as f:
    text = ' '.join(f.readlines())

messages = []
for message in text.split('От:'):
    if 'Имя:' in message:
        m = message.split('Имя:')[1]
        messages.append(m.replace('\n', ' '))

import pandas as pd

df = pd.DataFrame(messages, columns=['text'])
df['label'] = 1
df

,text,label
0,Davidkax Телефон: 81918381482 Сообщение...,1
1,JosephRib Телефон: 83995131172 Сообщени...,1
2,callievi11 Телефон: 82138886658 Сообщен...,1
3,Eddietet Телефон: 83816596797 Сообщение...,1
4,vidly Телефон: 81735583586 Сообщение: I...,1
...,...,...
215,Barrysip Телефон: 81899623474 Сообщение...,1
216,debbiecd3 Телефон: 89939737931 Сообщени...,1
217,AlbertCew Телефон: 89788629755 Сообщени...,1
218,Patrickkqv Телефон: 87423696752 Сообщен...,1


In [17]:
with open('data/not-spam.txt', encoding='cp1251') as f:
    text = ' '.join(f.readlines())

messages = []
for message in text.split('От:'):
    if 'Имя:' in message:
        m = message.split('Имя:')[1]
        messages.append(m.replace('\n', ' '))

import pandas as pd

df2 = pd.DataFrame(messages, columns=['text'])
df2['label'] = 0

In [18]:
df = pd.concat([df, df2])

In [20]:
df['label'].value_counts()

1    220
0    112
Name: label, dtype: int64

In [21]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm, trange

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader
from datasets import Dataset

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [23]:
data = Dataset.from_dict({'text': df['text'], 'label': df['label']}).train_test_split(test_size=0.2, seed=1)

In [24]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 265
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 67
    })
})

In [25]:
base_model = 'DeepPavlov/rubert-base-cased-conversational'

In [26]:
tokenizer = AutoTokenizer.from_pretrained(base_model)

In [27]:
data_tokenized = data.map(lambda x: tokenizer(x['text'], truncation=True, max_length=512), batched=True, remove_columns=['text'])

Map:   0%|          | 0/265 [00:00<?, ? examples/s]

Map:   0%|          | 0/67 [00:00<?, ? examples/s]

In [28]:
data_tokenized

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 265
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67
    })
})

In [32]:
print(data_tokenized['train'][260])

{'label': 0, 'input_ids': [101, 43302, 3355, 359, 476, 130, 30561, 156, 2987, 1873, 2675, 168, 30561, 132, 7411, 16655, 156, 13154, 6308, 97191, 24610, 6299, 44211, 156, 7963, 106, 10788, 18548, 32939, 166, 1188, 886, 2101, 1792, 166, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [33]:
collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [54]:
train_dataloader = DataLoader(data_tokenized['train'], shuffle=True, batch_size=4, collate_fn=collator)
val_dataloader = DataLoader(data_tokenized['test'], shuffle=False, batch_size=4, collate_fn=collator)
from torch.optim import Adam
model = AutoModelForSequenceClassification.from_pretrained(base_model, num_labels=2)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased-conversational were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassi

In [55]:
import torch

class BertClassifierSimple(torch.nn.Module):
    def __init__(self, num_labels):
        super(BertClassifierSimple, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = torch.nn.Dropout(self.bert.config.dropout)
        self.out = torch.nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        bert_output = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        output = self.out(self.dropout(bert_output[1]))  # output raw scores to be put into a softmax transformation
        return output

In [56]:
if torch.cuda.is_available():
    model.cuda()

In [57]:
# model.classifier.parameters()
optimizer = Adam(model.parameters(), lr=1e-5)  # with tiny batches, LR should be very small as well

In [58]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [59]:
losses = []
for epoch in trange(3):
    pbar = tqdm(train_dataloader)
    model.train()
    for i, batch in enumerate(pbar):
        out = model(**batch.to(model.device))
        out.loss.backward()
        if i % 1 == 0:
            optimizer.step()
            optimizer.zero_grad()
        losses.append(out.loss.item())
        pbar.set_description(f'loss: {np.mean(losses[-100:]):2.2f}')
    model.eval()
    eval_losses = []
    eval_preds = []
    eval_targets = []
    for batch in tqdm(val_dataloader):
        with torch.no_grad():
                out = model(**batch.to(model.device))
        eval_losses.append(out.loss.item())
        eval_preds.extend(out.logits.argmax(1).tolist())
        eval_targets.extend(batch['labels'].tolist())
    print('recent train loss', np.mean(losses[-100:]), 'eval loss', np.mean(eval_losses), 'accuracy', np.mean(np.array(eval_targets) == eval_preds))

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

recent train loss 0.24865837584235775 eval loss 0.08497256719890763 accuracy 0.9850746268656716


  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

recent train loss 0.05062259435653686 eval loss 0.09788846618988935 accuracy 0.9701492537313433


  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

recent train loss 0.010068924026563763 eval loss 0.10251752298106165 accuracy 0.9701492537313433


In [60]:
model.eval()
eval_losses = []
eval_preds = []
eval_targets = []
for batch in tqdm(val_dataloader):
    with torch.no_grad():
            out = model(**batch.to(model.device))
    eval_losses.append(out.loss.item())
    eval_preds.extend(out.logits.argmax(1).tolist())
    eval_targets.extend(batch['labels'].tolist())
print('recent train loss', np.mean(losses[-100:]), 'eval loss', np.mean(eval_losses), 'accuracy', np.mean(np.array(eval_targets) == eval_preds))

  0%|          | 0/17 [00:00<?, ?it/s]

recent train loss 0.010068924026563763 eval loss 0.10251752298106165 accuracy 0.9701492537313433


In [61]:
from sklearn.metrics import confusion_matrix

confusion_matrix(eval_targets, eval_preds)

array([[21,  1],
       [ 1, 44]])

In [62]:
model.save_pretrained('models/spam_classifier')
tokenizer.save_pretrained('models/spam_classifier')

('models/spam_classifier/tokenizer_config.json',
 'models/spam_classifier/special_tokens_map.json',
 'models/spam_classifier/vocab.txt',
 'models/spam_classifier/added_tokens.json',
 'models/spam_classifier/tokenizer.json')

In [63]:
model = AutoModelForSequenceClassification.from_pretrained('models/spam_classifier')
tokenizer = AutoTokenizer.from_pretrained('models/spam_classifier')

In [64]:
def classify(text):
    with torch.no_grad():
        proba = torch.softmax(model(**tokenizer(text, return_tensors='pt', truncation=True, max_length=512).to(model.device)).logits, -1)
    return proba.cpu().numpy()[0]

In [65]:
with open('data/val-spam.txt', encoding='cp1251') as f:
    text = ' '.join(f.readlines())

messages = []
for message in text.split('От:'):
    if 'Имя:' in message:
        m = message.split('Имя:')[1]
        messages.append(m.replace('\n', ' '))

messages

[' hotplayerMypog   Телефон: 86858628552     Сообщение: Популярный музыкальный портал предлагает послушать красивую, романтическую и нежную музыку, которая помогает взбодриться, улучшить настроение, развеяться в будний день после активной работы. Если и вы находитесь в поисках качественной музыки, которую можно не только поставить в машину, но и скачать на любое устройство для веселья, то скорей заходите на сайт https://hotplayer.ru (лезгинка терских казаков <https://hotplayer.ru/?s=лезгинка>  ) на котором собраны самые лирические треки современности. Их можно слушать без остановки, нон-стоп и получать от этого много удовольствия. Музыка уносит в удивительный, невероятный мир, в котором вам будет спокойно и гармонично.     Здесь представлены как давно полюбившиеся хиты прошлых лет, так и многообещающие новинки. Их выхода ждали давно и с особым нетерпением, а потому вы сможете начать прослушивать любимые треки и радоваться их качественному, необычному и интересному звучанию. И самое гла

In [69]:
for m in messages:
    print(m[:100], 'prob=', 'SPAM' if classify(m)[1] > 0.5  else 'NOT-SPAM')

 hotplayerMypog   Телефон: 86858628552     Сообщение: Популярный музыкальный портал предлагает послу prob= SPAM
 zorovjedo   Телефон: 86295593619     Сообщение: Восстановление желудка  и иные знания в разделе о З prob= SPAM
 gilbertjq11   Телефон: 85483735439     Сообщение: Hot photo galleries blogs and pictures  http://da prob= SPAM
 JosephDig   Телефон: 85988691678     Сообщение: Est error molestiae in dolorem similique. Ipsum ips prob= SPAM
 JeroldPef   Телефон: 89113199118     Сообщение: Лжеюристы, кто они?   Гк защита граждан контакты    prob= SPAM


In [71]:
classify('Елена   Телефон: 89101455516    Сообщение: Интересует занятие Читайка с мамами')

array([0.9950984 , 0.00490153], dtype=float32)